# <span style='font-family: CMU Sans Serif, sans-serif;'> Feed-forward neural networks  </span> 

From the high level deep neural networks use composition of a series of non-linear function to model non-linearity
$$
\mathbf{h}^{(L)} = \mathbf{g}^{(L)} \circ \mathbf{g}^{(L-1)} \circ \cdots \circ \mathbf{g}^{(1)} (\mathbf{x})
$$
where $\circ$ is the composition of two functions and $L$ is the number of hidden layers (depth). 

If we let $\mathbf{h}^{(0)} \triangleq \mathbf{x}$ we can recursively define $\mathbf{h}^{(\ell)} = \mathbf{g}^{(\ell)}(\mathbf{h}^{(\ell - 1)})$ for all $\ell = 1,2, \ldots ,L$. 

Feed-forward neural networks are a type of neural network with a specific type of $\mathbf{g}^{(\ell)}$: for $\ell = 1,2, \ldots ,L$ we define
$$
\mathbf{h}^{(\ell)} = \mathbf{g}^{(\ell)}(\mathbf{h}^{(\ell-1)}) \triangleq \boldsymbol{\sigma} (\mathbf{W}^{(\ell)} \mathbf{h}^{(\ell - 1)} + \mathbf{b}^{\ell}),
$$
where $\mathbf{W}^{(\ell)}$ and $\mathbf{b}^{(\ell)}$ are weights and biases, respectively, associated with the $l$-th layer, and $\boldsymbol{\sigma}(\cdot ) $ is the activation function.

ReLU given by $\sigma(z) = \max\{ z,0 \}$ is a popular activation function, since its derivative is either $1$ or $0$, making training more efficient. 

Given an output $\mathbf{h}^{(L)}$ from the final hidden layer and a label $y$ we can define a loss function to minimize. 

## <span style='font-family: CMU Sans Serif, sans-serif;'> Back-propagation in computational graphs  </span> 

Training seeks to minimize the loss ovre all the training date. We can minimize with *stochastic gradient descent* (SDG), amongst others like mini-batch or simple batch gradient descent.

SGD starts from an initial value $\boldsymbol{\theta}^{0} $ and updates this in the direction of most negative gradient descent creating $\boldsymbol{\theta}^{t}$.

The difference is that in each update a small subsample $\mathcal{B} \sub [n]$ called a mini-batch -- typically powers of $2$ between $32$ and $512$ -- randomly drawn from the training sample, and the gradient calculation is only on $\mathcal{B}$ instead of the full batch $[n]$. This saves a lot of time computationally. A pass of the entire training set is called an **epoch**. 

The key to the above training procedure is the calucaltion of the gradient, $\nabla \ell_{\mathcal{B}}(\boldsymbol{\theta})$, where
$$
\ell_{\mathcal{B}} \triangleq \left| \mathcal{B} \right|^{-1} \sum_{i \in \mathcal{B}} \mathcal{L}(\mathbf{f}(\mathbf{x}_i;\boldsymbol{\theta}), y_i)
$$

The computational savings stem from the fact $\mathcal{B}$ is much smaller than $n$. Gradient computation, however, is in general non-trivial for complex models, and it is susceptible to numerical instability for a model with large depth.

<div style='border: 1px solid currentColor; border-radius: 2px; padding: 10px; width: 95%;'>

<span style='font-family: CMU Sans Serif, sans-serif; font-weight: bold; font-size: 17px;'>
Back-propagation 
</span> 

This is an efficient algorithm that is a direct application of the chain rule. The calculation is performed back-wards: first calcualte $\frac{\partial \ell_{\mathcal{B}}}{\partial \mathbf{h}^{(L)}}$, then $\frac{\partial \ell_{\mathcal{B}}}{\partial \mathbf{h}^{(L-1)}}$ and then $\frac{\partial \ell_{\mathcal{B}}}{\partial \mathbf{h}^{(1)}}$. 

Consider a ReLU activation function, then we have the recursive/backward relation
$$
\frac{\partial \ell_{\mathcal{B}}}{\partial \mathbf{h}^{(\ell - 1)}} = \frac{\partial \mathbf{h}^{(\ell)}}{\partial \mathbf{h}^{(\ell - 1)}} \cdot \frac{\partial \ell_{\mathcal{B}}}{\partial \mathbf{h}^{(\ell)}} = (\mathbf{W}^{(\ell)})^{\top} \operatorname{diag}\left( \boldsymbol{1} \{ \mathbf{W}^{(\ell)} \mathbf{h}^{(\ell - 1)} + \mathbf{b}^{(\ell)} \ge \boldsymbol{0}  \}  \right) \frac{\partial \ell_{\mathcal{B}}}{\partial \mathbf{h}^{(\ell)}}
$$

where $\operatorname{diag}(\cdot )$ denotes a diagonal matrix with element given by the argument and the indicator is the derivative of the ReLU function.
We note that the calculate of the derivative above implies the calculation of the derivative from the next layer, which is why we start at the last layer and work backwards through the network.

These derivatives are then used to update the parameters. For instance, the gradient update for $\mathbf{W}^{(\ell)}$ is given by
$$
\mathbf{W}^{(\ell)} \leftarrow \mathbf{W}^{(\ell)} - \eta \frac{\partial \ell_{\mathcal{B}}}{\partial \mathbf{W}^{(\ell)}}, \quad \text{where} \quad \frac{\partial \ell_{\mathcal{B}}}{\partial W^{(\ell)}_{jm}} = \frac{\partial \ell_{\mathcal{B}}}{\partial h_j^{(\ell)}} \cdot  \sigma' \cdot h_{m}^{(\ell-1)}
$$

where $\sigma'=1$ if the $j$-th element of $\mathbf{W}^{(\ell)}\mathbf{h}^{(\ell - 1)} + \mathbf{b}^{(\ell)}$ is non-negative and $\sigma' = 0$ otherwise. $\eta > 0$ is the learning rate.
</div>

A generel way to think about neural network models is to consider computational graphs; acyclic graph representing the functional relationships between variables. Consider an MLP with a single hidden layer and an $\ell_{2}$ regularization
$$
\ell_{\mathcal{B}}^{\lambda}(\boldsymbol{\theta} ) = \ell_{\mathcal{B}}(\boldsymbol{\theta} ) + r_{\lambda}(\boldsymbol{\theta} ) = \ell_{\mathcal{B}}(\boldsymbol{\theta} ) + \lambda \left( \sum_{j,j'}(W_{j,j'}^{(1)})^{2} +   \sum_{j,j'}(W_{j,j'}^{(2)})^{2} \right) ,
$$
where $\ell_{\mathcal{B}}(\boldsymbol{\theta}) $ is defined above, $\lambda \ge 0$ is a tuning parameter.

### <span style='font-family: CMU Sans Serif, sans-serif;'> Computational graphs  </span> 

Below we illustrate computational graphs. The term $\ell_{\mathcal{B}}(\boldsymbol{\theta})$ is the result of 4 compositions: the first is the input data $\mathbf{x}$ which is multiplied with the weight matrix $\mathbf{W}^{(1)}$ resulting in $\mathbf{u}^{(1)}$, then it goes through the ReLU activation function resulting in $\mathbf{h}^{(1)}$, then it multiplies another weight matrix $\mathbf{W}^{(2)}$ leading to $\mathbf{p}$ and finally it produces the cross-entropy (or similar loss functions) with the label $y$. The regularization term is calculated similarly.

A forward pass is done when all the nodes are evaluated starting from the input $\mathbf{x}$. A backwards pass then calculates the gradients of $\ell_{\mathcal{B}}^{\lambda}$ with respect to all other nodes in the reverse direction. Due to the chain rule, the gradient calculation for a variable (e.g., $\frac{\partial \ell_{\mathcal{B}}}{\partial \mathbf{u}^{(1)}}$) is simple: it only depends on the gradient value of the variables ($\frac{\partial \ell_{\mathcal{B}}}{\partial \mathbf{h}}$) the current node points to, and the function derivative evaluated at the current variable value ($\mathbf{\sigma}' (\mathbf{u}^{(1)}$). Thus, each iteration, a computation graph only needs to (1) calculate and store the function evaluations at each node in the forward pass, and then (2) calculate all derivatives in the backwards pass.